# NVIDIA NeMo 모델 커스터마이제이션 종합 실습

이 노트북은 NeMo Microservices를 사용한 LLM 평가 및 미세 조정의 전체 워크플로우를 실습합니다.

## 목차
1. [환경 설정](#1.-환경-설정)
2. [NIM 기본 평가](#2.-NIM-기본-평가)
3. [데이터셋 준비](#3.-데이터셋-준비)
4. [Zero-Shot vs. ICL 비교](#4.-Zero-Shot-vs.-ICL-비교)
5. [LoRA 미세 조정](#5.-LoRA-미세-조정)
6. [결과 비교 및 분석](#6.-결과-비교-및-분석)

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치 및 Import

In [ ]:
# 필요한 패키지 설치
!pip install -q requests datasets bs4 ftfy huggingface_hub matplotlib pandas

print("✅ 패키지 설치 완료")

In [ ]:
import os
import re
import json
import time
import subprocess
from pprint import pprint
from typing import Dict, List

import requests
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML

# 데이터셋 처리
from datasets import Dataset, load_dataset
from bs4 import BeautifulSoup
import ftfy

# Hugging Face Hub
from huggingface_hub import HfApi

print("✅ 라이브러리 import 완료")

### 1.2 NeMo Microservices 엔드포인트 설정

In [ ]:
# NeMo Microservices 엔드포인트
datastore_url = "http://nemo-datastore.local"
nim_url = "http://llama3-2-3b-instruct.local"
eval_url = "http://nemo-evaluator.local"
customizer_url = "http://nemo-customizer.local"
entitystore_url = "http://nemo-entity-store.local"

# 모델 ID
NIM_model_id = "meta/llama-3.2-3b-instruct"

# MLflow 설정
mlflow_uri = "http://localhost:30090"

# NVIDIA API 설정
NGC_API_KEY = os.environ.get("NGC_API_KEY", "nvapi-mock-key")
BASE_URL = os.environ.get("NVIDIA_BASE_URL", "http://proxy/v1/chat/completions")

print("✅ 엔드포인트 설정 완료")

### 1.3 서비스 상태 확인

In [ ]:
def check_service_health(name: str, url: str) -> bool:
    """서비스 상태 확인"""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {name}: 정상 작동")
            return True
        else:
            print(f"⚠️  {name}: 상태 코드 {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ {name}: 연결 실패 - {e}")
        return False


print("서비스 상태 확인 중...\n")
services = [
    ("NeMo Evaluator", f"{eval_url}/health"),
    ("NeMo Data Store", f"{datastore_url}/v1/health"),
    ("NeMo Entity Store", f"{entitystore_url}/v1/health/ready"),
    ("NeMo Customizer", f"{customizer_url}/health/ready"),
    ("NIM (Llama 3.2 3B)", f"{nim_url}/v1/health/ready"),
]

all_healthy = all(check_service_health(name, url) for name, url in services)

if all_healthy:
    print("\n🎉 모든 서비스가 정상 작동 중입니다!")
else:
    print("\n⚠️  일부 서비스가 작동하지 않습니다. 환경을 확인하세요.")

### 1.4 MLflow 설정

In [ ]:
# MLflow 포트 포워딩 설정
try:
    subprocess.Popen(
        [
            "kubectl",
            "-n",
            "mlflow",
            "port-forward",
            "--address",
            "0.0.0.0",
            "service/mlflow-tracking",
            "30090:80",
        ],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        close_fds=True,
    )
    print("✅ MLflow 포트 포워딩 시작됨")
    print(f"   MLflow UI: {mlflow_uri}")
except Exception as e:
    print(f"⚠️  MLflow 포트 포워딩 실패: {e}")

## 2. NIM 기본 평가

### 2.1 NIM 상태 확인 및 기본 추론

In [ ]:
# NIM 대기
print("NIM 서비스 준비 대기 중...")
while True:
    try:
        response = requests.get(f"{nim_url}/v1/health/ready")
        if response.status_code == 200:
            clear_output(wait=True)
            print("✅ NIM 서비스가 준비되었습니다!")
            break
        else:
            clear_output(wait=True)
            print(f"대기 중... (상태 코드: {response.status_code})")
    except:
        clear_output(wait=True)
        print("대기 중... (연결 시도)")
    time.sleep(10)

In [ ]:
# 배포된 모델 확인
response = requests.get(f"{nim_url}/v1/models")
models = response.json()["data"]

print("배포된 모델:")
for model in models:
    print(f"  - {model['id']}")

In [ ]:
# 기본 추론 테스트
def query_nim(prompt: str, model: str = NIM_model_id, max_tokens: int = 100) -> str:
    """NIM에 쿼리하여 응답 받기"""
    request_body = {
        "model": model,
        "messages": [
            {"role": "system", "content": "Be succinct in your response."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": 0.2,
    }
    
    response = requests.post(f"{nim_url}/v1/chat/completions", json=request_body)
    return response.json()["choices"][0]["message"]["content"]


# 테스트 쿼리
test_prompt = "What is the capital of Spain?"
response = query_nim(test_prompt)

print(f"질문: {test_prompt}")
print(f"답변: {response}")

### 2.2 GSM8K 벤치마크 평가

In [ ]:
%%bash
# GSM8K 벤치마크 평가 (10개 샘플)
lm_eval --model local-completions \
        --tasks gsm8k \
        --output_path ./eval_results/gsm8k \
        --model_args model=meta/llama-3.2-3b-instruct,base_url=http://llama3-2-3b-instruct.local/v1/completions \
        --limit 10 | awk -F'|' '/acc/ {gsub(/^ +| +$/, "", $8); print "GSM8K Accuracy:", $8}'

### 2.3 LLM-as-a-Judge 평가

In [ ]:
# Faithfulness 프롬프트 정의
prompt_faithfulness = """
System: You are an impartial judge assessing the faithfulness of an AI-generated response.

Evaluation Criteria:
1. Correctness: Does the response correctly reflect facts from the ground truth?
2. No Hallucination: Does it introduce false information?
3. Completeness: Does it omit key facts?
4. Paraphrase Accuracy: Is the meaning preserved?

Provide a faithfulness score from 1 (poor) to 5 (perfect) with brief explanation.
"""

# 테스트 예제
question = "What are the three most populated countries worldwide?"
ground_truth = "India, China and the United States"
generated_answer = "India, China and Spain"  # 의도적 오류

# LLM-as-a-Judge로 평가
judge_response = query_nim(
    prompt=f"{prompt_faithfulness}\n\nQuestion: {question}\nGround truth: {ground_truth}\nAnswer: {generated_answer}",
    max_tokens=200
)

print("LLM-as-a-Judge 평가 결과:")
print(judge_response)

## 3. 데이터셋 준비

### 3.1 법률 데이터셋 다운로드 및 전처리

In [ ]:
# Law StackExchange 데이터셋 로드
print("데이터셋 다운로드 중...")
hf_ds = load_dataset("ymoslem/Law-StackExchange")
hf_ds = hf_ds["train"].remove_columns(
    ["link", "license", "question_id", "answers", "tags"]
)

print(f"전체 데이터셋 크기: {len(hf_ds)} 샘플")
print("\n샘플 예시:")
pprint(hf_ds[0])

In [ ]:
# 데이터 필터링 (고품질 샘플 선별)
print("데이터 필터링 중...")
filtered_ds = hf_ds.filter(lambda row: row["score"] > 10)
filtered_ds = filtered_ds.filter(
    lambda row: len(row["question_body"]) >= 50 and len(row["question_body"]) <= 1000
)

print(f"필터링 후 크기: {len(filtered_ds)} 샘플")

In [ ]:
# HTML 태그 제거 및 텍스트 정리
def clean_text(text: str) -> str:
    """HTML 태그 제거 및 텍스트 정규화"""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"\s+", " ", text).strip()
    text = ftfy.fix_text(text)
    return text


print("텍스트 정리 중...")
modified_ds = filtered_ds.map(
    lambda row: {
        "question_body": clean_text(row["question_body"]),
        "question_title": clean_text(row["question_title"]),
    }
)

print("\n정리 후 샘플:")
pprint(modified_ds[0])

In [ ]:
# NeMo 형식으로 변환
nemo_ds = (
    modified_ds.rename_column("question_body", "prompt")
    .rename_column("question_title", "completion")
    .remove_columns(["score"])
)

print("NeMo 형식 변환 완료")
print(f"최종 데이터셋: {nemo_ds}")

### 3.2 데이터셋 분할

In [ ]:
# 훈련/검증/테스트 분할
ds = nemo_ds.train_test_split(test_size=0.5, shuffle=False)
training_dataset = ds["train"]

ds = ds["test"].train_test_split(test_size=0.1, shuffle=False)
validation_dataset = ds["train"]
evaluation_dataset = ds["test"]

print("데이터셋 분할 완료:")
print(f"  훈련: {len(training_dataset)} 샘플")
print(f"  검증: {len(validation_dataset)} 샘플")
print(f"  테스트: {len(evaluation_dataset)} 샘플")

### 3.3 Data Store에 업로드

In [ ]:
# 데이터셋 저장
dataset_name = "legal_dataset"
os.makedirs("./datasets", exist_ok=True)

training_dataset.to_json("./datasets/training.jsonl")
validation_dataset.to_json("./datasets/validation.jsonl")
evaluation_dataset.to_json("./datasets/testing.jsonl")

print("✅ 로컬에 데이터셋 저장 완료")

In [ ]:
# HuggingFace Hub을 통해 NeMo Data Store에 업로드
try:
    api = HfApi(endpoint=datastore_url)
    repo_id = f"default/{dataset_name}"
    
    # 훈련 데이터 업로드
    api.upload_file(
        path_or_fileobj="./datasets/training.jsonl",
        path_in_repo="training/training.jsonl",
        repo_id=repo_id,
        repo_type="dataset"
    )
    
    # 검증 데이터 업로드
    api.upload_file(
        path_or_fileobj="./datasets/validation.jsonl",
        path_in_repo="validation/validation.jsonl",
        repo_id=repo_id,
        repo_type="dataset"
    )
    
    # 테스트 데이터 업로드
    api.upload_file(
        path_or_fileobj="./datasets/testing.jsonl",
        path_in_repo="testing/testing.jsonl",
        repo_id=repo_id,
        repo_type="dataset"
    )
    
    print(f"✅ NeMo Data Store에 업로드 완료: {repo_id}")
except Exception as e:
    print(f"⚠️  업로드 실패: {e}")
    print("   로컬 파일을 사용합니다.")

## 4. Zero-Shot vs. ICL 비교

### 4.1 Few-Shot 예제 준비

In [ ]:
# Few-Shot 예제 선택 (ICL용)
few_shot_examples = [
    {
        "question": "Do police officers have to stop an interrogation when right to counsel has been invoked?",
        "title": "Must the interrogation stop when the right to counsel has been invoked by a suspect?"
    },
    {
        "question": "What constitutes workplace discrimination under employment law?",
        "title": "Workplace Discrimination: Legal Definitions and Protected Characteristics"
    },
    {
        "question": "How does copyright law apply to software and source code?",
        "title": "Copyright Protection for Software: Source Code and Object Code"
    },
]

# Few-Shot 프롬프트 생성
def create_few_shot_prompt(question: str, examples: List[Dict]) -> str:
    """Few-Shot 프롬프트 생성"""
    prompt = "Generate a concise, engaging title for the following legal questions.\n\n"
    
    # 예제 추가
    for i, ex in enumerate(examples, 1):
        prompt += f"Example {i}:\n"
        prompt += f"QUESTION: {ex['question']}\n"
        prompt += f"TITLE: {ex['title']}\n\n"
    
    # 실제 질문
    prompt += f"QUESTION: {question}\nTITLE:"
    
    return prompt

print("✅ Few-Shot 프롬프트 생성 함수 준비 완료")

### 4.2 Zero-Shot vs. Few-Shot 비교 평가

In [ ]:
# 테스트 샘플 선택
test_sample = evaluation_dataset[0]
test_question = test_sample["prompt"]
ground_truth = test_sample["completion"]

print("="*80)
print("테스트 질문:")
print(test_question[:200] + "...")
print("\n정답 (Ground Truth):")
print(ground_truth)
print("="*80)

In [ ]:
# Zero-Shot 추론
zero_shot_prompt = f"Generate a concise, engaging title for the following legal question.\n\nQUESTION: {test_question}\nTITLE:"

zero_shot_response = query_nim(zero_shot_prompt, max_tokens=50)

print("\n🔹 Zero-Shot 결과:")
print(zero_shot_response)

In [ ]:
# Few-Shot (ICL) 추론
few_shot_prompt = create_few_shot_prompt(test_question, few_shot_examples)

few_shot_response = query_nim(few_shot_prompt, max_tokens=50)

print("\n🔹 Few-Shot (ICL) 결과:")
print(few_shot_response)

In [ ]:
# 결과 비교
print("\n" + "="*80)
print("결과 요약")
print("="*80)
print(f"Ground Truth:  {ground_truth}")
print(f"Zero-Shot:     {zero_shot_response}")
print(f"Few-Shot (ICL): {few_shot_response}")
print("="*80)

## 5. LoRA 미세 조정

### 5.1 사용 가능한 모델 설정 확인

In [ ]:
# Customizer에서 지원하는 모델 확인
response = requests.get(
    f"{customizer_url}/v1/customization/configs",
    headers={"accept": "application/json"},
    timeout=30
)

print("지원되는 모델 설정:")
print(json.dumps(response.json(), indent=2))

### 5.2 LoRA 미세 조정 작업 시작

In [ ]:
# LoRA 미세 조정 설정
training_config = {
    "config": "meta/llama-3.2-3b-instruct",
    "dataset": {"name": dataset_name},
    "hyperparameters": {
        "training_type": "sft",  # Supervised Fine-Tuning
        "finetuning_type": "lora",
        "epochs": 3,
        "batch_size": 32,
        "learning_rate": 1e-4,
        "lora": {
            "adapter_dim": 8,      # LoRA rank
            "adapter_dropout": 0.1,
        },
    },
}

print("미세 조정 설정:")
pprint(training_config)

In [ ]:
# 미세 조정 작업 시작
response = requests.post(
    f"{customizer_url}/v1/customization/jobs",
    headers={"Content-Type": "application/json"},
    json=training_config,
    timeout=30
)

if response.status_code == 200:
    job_data = response.json()
    job_id = job_data["id"]
    print(f"✅ 미세 조정 작업 시작됨: {job_id}")
else:
    print(f"❌ 작업 시작 실패: {response.status_code}")
    print(response.text)
    job_id = None

### 5.3 학습 진행 모니터링

In [ ]:
# 학습 진행 상황 모니터링
if job_id:
    import sys
    
    status = "initializing"
    refresh_interval = 30
    
    print("학습 진행 상황 모니터링 중...\n")
    
    while status in {"initializing", "running", "created"}:
        job = requests.get(
            f"{customizer_url}/v1/customization/jobs/{job_id}",
            timeout=10
        ).json()
        
        status = job["status"]
        
        # 카운트다운 타이머로 상태 표시
        for remaining in range(refresh_interval, 0, -1):
            sys.stdout.write('\033[K')  # 라인 지우기
            sys.stdout.write(f"\r⏳  상태: {status:<12} (다음 확인: {remaining}초)")
            sys.stdout.flush()
            time.sleep(1)
    
    # 최종 상태
    sys.stdout.write('\033[K')
    print(f"\n\n🏁  작업 완료 → {job['status']}")
    
    if job['status'] == 'completed':
        output_model = job["output_model"]
        print(f"\n✅ 미세 조정된 모델: {output_model}")
    else:
        print(f"\n❌ 작업 실패: {job.get('status_details', {}).get('error_message', 'Unknown error')}")

### 5.4 학습 메트릭 시각화

In [ ]:
# 학습 메트릭 시각화
if job_id and job.get('status') == 'completed':
    metrics = job['status_details']['metrics']['metrics']
    
    # Training & Validation Loss
    train_loss = pd.DataFrame(metrics['train_loss'])
    val_loss = pd.DataFrame(metrics['val_loss'])
    
    # 요약 통계
    print("\n" + "="*80)
    print("학습 요약")
    print("="*80)
    print(f"최종 Training Loss:  {train_loss['value'].iloc[-1]:.4f}")
    print(f"최적 Training Loss:  {train_loss['value'].min():.4f}")
    print(f"최적 Validation Loss: {val_loss['value'].min():.4f}")
    print(f"총 학습 스텝:        {train_loss['step'].max()}")
    print(f"완료된 Epoch:        {job['status_details']['epochs_completed']}")
    print(f"최적 Epoch:          {job['status_details']['best_epoch']}")
    print("="*80)
    
    # 그래프 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(train_loss['step'], train_loss['value'], label='Training Loss', linewidth=2)
    plt.scatter(val_loss['step'], val_loss['value'], color='red', label='Validation Loss', s=50)
    plt.xlabel('Training Steps', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('LoRA 미세 조정 진행 상황', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. 결과 비교 및 분석

### 6.1 미세 조정 모델 테스트

In [ ]:
# 미세 조정 모델로 추론
if 'output_model' in locals():
    lora_prompt = f"Generate a concise, engaging title for the following legal question.\n\nQUESTION: {test_question}\nTITLE:"
    
    lora_response = query_nim(lora_prompt, model=output_model, max_tokens=50)
    
    print("\n🔹 LoRA 미세 조정 모델 결과:")
    print(lora_response)
else:
    print("미세 조정 모델이 준비되지 않았습니다.")
    lora_response = "(미세 조정 미완료)"

### 6.2 종합 비교

In [ ]:
# 결과 종합 비교
comparison_df = pd.DataFrame({
    "접근 방식": ["Ground Truth", "Zero-Shot", "Few-Shot (ICL)", "LoRA Fine-tuned"],
    "생성된 제목": [
        ground_truth,
        zero_shot_response,
        few_shot_response,
        lora_response
    ]
})

print("\n" + "="*100)
print("종합 비교 결과")
print("="*100)
display(comparison_df)
print("="*100)

### 6.3 성능 메트릭 비교 (가상)

In [ ]:
# 성능 메트릭 비교 차트 (예시)
performance_data = {
    "접근 방식": ["Zero-Shot", "Few-Shot (ICL)", "LoRA Fine-tuned"],
    "ROUGE Score": [0.35, 0.52, 0.68],
    "BLEU Score": [0.28, 0.45, 0.61],
    "F1 Score": [0.31, 0.48, 0.64],
}

perf_df = pd.DataFrame(performance_data)

# 막대 그래프
fig, ax = plt.subplots(figsize=(12, 6))
x = range(len(perf_df["접근 방식"]))
width = 0.25

ax.bar([i - width for i in x], perf_df["ROUGE Score"], width, label='ROUGE Score')
ax.bar(x, perf_df["BLEU Score"], width, label='BLEU Score')
ax.bar([i + width for i in x], perf_df["F1 Score"], width, label='F1 Score')

ax.set_xlabel('접근 방식', fontsize=12)
ax.set_ylabel('점수', fontsize=12)
ax.set_title('평가 메트릭 비교 (예시)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(perf_df["접근 방식"])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n성능 메트릭 표:")
display(perf_df)

### 6.4 결론 및 권장 사항

In [ ]:
print("""
==================================================================================
결론 및 권장 사항
==================================================================================

1. Zero-Shot
   장점: 즉시 사용 가능, 추가 리소스 불필요
   단점: 도메인 특화 작업에서 성능 제한적
   적용: 일반적인 질문, 프로토타이핑

2. Few-Shot (ICL)
   장점: 빠른 적응, 모델 수정 불필요
   단점: 컨텍스트 창 크기 제한, 임시적 개선
   적용: 중간 복잡도 작업, 빠른 실험

3. LoRA 미세 조정
   장점: 최고 성능, 영구적 개선, 효율적 학습
   단점: 학습 시간 필요, 고품질 데이터 필요
   적용: 프로덕션 배포, 도메인 특화 애플리케이션

권장 워크플로우:
1. Zero-Shot으로 베이스라인 성능 확인
2. Few-Shot으로 빠른 성능 개선 테스트
3. 프로덕션 배포 전 LoRA 미세 조정 수행
4. MLflow로 모든 실험 추적 및 비교
==================================================================================
""")

## 마무리

### 리소스 정리

In [ ]:
# 사용한 리소스 정리 (선택 사항)
cleanup = input("리소스를 정리하시겠습니까? (y/n): ")

if cleanup.lower() == 'y':
    print("\n리소스 정리 중...")
    
    # 임시 파일 삭제
    import shutil
    if os.path.exists("./datasets"):
        shutil.rmtree("./datasets")
    if os.path.exists("./eval_results"):
        shutil.rmtree("./eval_results")
    
    print("✅ 로컬 파일 정리 완료")
    print("\nKubernetes 리소스를 정리하려면 다음 명령을 실행하세요:")
    print("  kubectl delete namespace llama3-2-3b-instruct")
else:
    print("\n리소스가 유지됩니다.")